# Scraping and downloading Inside Airbnb links to Pandas

In [1]:
import copy
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.set_option('max_colwidth', 100)
pd.set_option('max_columns',500)
import logging
import boto3
from botocore.exceptions import ClientError
import os

cities = ['denver','portland','columbus','chicago','pacific-grove']

url = 'http://insideairbnb.com/get-the-data.html'
r = requests.get(url)
print(r.status_code)

200


## Upload Functions
- upload_file: upload a new file to aws s3 bucket
- get_links: crawls the inside airbnb site and downloads links
- create_link_dfs: creates a separate dataframe of links for each dataset
- load_df: downloads csv from each link for the listed datasets

In [2]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name=None)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def get_links(city_list):
    """
    Description: Finds links in beautiful soup and appends urls to list
    params: list of cities for which user wants to download data
    output: list of urls
    """
    soup = BeautifulSoup(r.content, "lxml")
    links = []
    for city in city_list:
        table = soup.find('table',{'class':f'table table-hover table-striped {city}'})
        for row in table.find_all('a'):
            links.append(row['href']) if 'listings' in row['href'] else \
            links.append(row['href']) if 'reviews' in row['href'] else 0
    return links

def create_link_dfs():
    """
    no params:
    purpose: takes inside airbnb links, adds them to pandas, and transforms them 
    into new dfs split by file type.
    output: df (primary), and splits df1,df2,df3,df4
    """
    links = get_links(cities)
    df = pd.DataFrame(links,columns = ['link'])
    #city_stop_index = df.link.str.slice(46).str.index('/')
    df_split = df['link'].str.split('/').apply(pd.Series)\
                         .rename(columns = lambda x: 'link_' + str(x))[['link_5',
                                                                        'link_6',
                                                                        'link_8']]
    df = df.join(df_split).rename(columns ={'link':'link',
                                            'link_5':'city',
                                            'link_6':'date',
                                            'link_8':'file'})
    
    df1 = df[df['file'] == 'listings.csv.gz'].reset_index().drop('index',axis=1)
    df2 = df[df['file'] == 'listings.csv'].reset_index().drop('index',axis=1)
    df3 = df[df['file'] == 'reviews.csv.gz'].reset_index().drop('index',axis=1)
    df4 = df[df['file'] == 'reviews.csv'].reset_index().drop('index',axis=1)
    
    return df,df1,df2,df3,df4


def load_df(load_count,df_file):
    """
    description: loads csvs and gzips for each df_file type
    params: count: how many urls shoudl be loaded and concatenated into DF
    params: df_file: which of 4 inside airbnb files need downloading
    output: new df, list of url indices that failed to download
    """
    
    missed_index = []
    columns = ['id','city','country_code','country','review_scores_rating','summary']
    df1 = pd.read_csv(df_file.reset_index().drop('index',axis=1).iloc[0][0],compression='gzip')[columns]
    df1['scrape_date'] = pd.to_datetime(df_file.reset_index().drop('index',axis=1).iloc[0]['date'])
    for i in range(1,load_count):
        #print('Test_{}'.format(i),'/ Total_{}'.format(load_count),' {}'.format(df_file.reset_index().drop('index',axis=1).iloc[i]['city']))
        df2 = df1.copy()
        df = df1.copy()
        try:
            df1 = pd.read_csv(df_file.reset_index().drop('index',axis=1).iloc[i][0],compression='gzip')[columns]
            df1['city'] = df_file.reset_index().drop('index',axis=1).iloc[i]['city']
            df1['scrape_date'] = pd.to_datetime(df_file.reset_index().drop('index',axis=1).iloc[i]['date'])
            df1 = pd.concat([df,df1],sort=False,ignore_index=True)
        except:
            df1 = df2.copy()
            missed_index.append(i)
            #print('Fail_{}'.format(i),'/ Total_{}'.format(load_count))
            
    df1.to_pickle('/home/ubuntu/Notebooks/data/{}.pickle'.format('df1'))
    return df1,missed_index

### Creating DFs for further use

In [ ]:
df,df1,df2,df3,df4 = create_link_dfs()

num_loads1 = len(df1)
num_loads2 = len(df2)
num_loads3 = len(df3)
num_loads4 = len(df4)

listings_gz,list1 = load_df(len(df1.iloc[::3]),df1.iloc[::3])

# pd.read_csv(df1.reset_index().drop('index',axis=1).iloc[0][0],compression='gzip')['city']
# df1.reset_index().drop('index',axis=1).iloc[1]['city']
# pd.read_csv(df1.reset_index().drop('index',axis=1).iloc[0][0],compression='gzip')['city']
# df1.reset_index().drop('index',axis=1)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [110]:
print(listings_gz.shape)

(821327, 7)


In [22]:
listings_gz.shape

(133975, 7)

In [33]:
s3 = boto3.client('s3') 
s3.list_buckets()
s3.list_buckets()['Buckets']
for b in s3.list_buckets()['Buckets']:
    print(b['Name'])
    

response = s3.list_objects_v2(Bucket='b.riddle.001')
response['Contents']
for obj in response['Contents']:
    print(obj['Key'])

b.riddle.001
/Users/ginariddle/Desktop/g.school/data/salary_data.csv


In [22]:

upload_file('/Users/ginariddle/Desktop/g.school/data/salary_data.csv', 'b.riddle.001', object_name=None)

True